In [178]:
from __future__ import print_function

from PIL import Image, ImageFilter
import PIL
from IPython import display
from IPython.display import clear_output

import numpy as np

from matplotlib.pyplot import imshow
%matplotlib inline

In [180]:
import squeezedet as nn
net = nn.init()

INFO:tensorflow:Summary name mean iou is illegal; using mean_iou instead.


InternalError: Dst tensor is not initialized.
	 [[Node: save/RestoreV2_45/_75 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_77_save/RestoreV2_45", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]
	 [[Node: save/RestoreV2_52/_44 = _Send[T=DT_FLOAT, client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_47_save/RestoreV2_52", _device="/job:localhost/replica:0/task:0/cpu:0"](save/RestoreV2_52)]]

In [14]:
data_path = "./data/"
trace_path = "./trace/"
ext_pics = ".png"
ext_trace = ".txt"
num_pics = 400

In [15]:
def write_trace(file_name, (boxes,probs,cats)):
    """Store detections in file"""
    num_cars = cats.count(0)
    
    lines = ()
    for box, prob, cat in zip(boxes, probs, cats):
        # Store just cars
        if cat == 0:
            line = np.hstack((box,np.array((prob,cat))))
            lines += (line,)
    
    np.savetxt(file_name + '_' + str(num_cars).zfill(2) + ext_trace, lines, fmt="%f")        
    
def read_trace(file_name):
    """Read detections from file"""
    return np.loadtxt(file_name + ext_trace)  

In [101]:
display.set_matplotlib_close(True)
pics = []

# Run detections
for i in range(0,num_pics):
    clear_output(wait=True)
    #raw_input()
    
    input("Press Enter to continue...")
    
    file_name = str(i).zfill(10)
    
    pred = nn.classify(data_path + file_name + ext_pics,net) 
    display.display(Image.open("tmp.png"))
    
    #write_trace(trace_path + file_name,pred)

KeyboardInterrupt: 

# Parse trace

In [69]:
trace = []

for i in range(0,num_pics):
    file_name = str(i).zfill(10)
    trace.append(read_trace(trace_path + file_name))

/home/tommaso/anaconda3/envs/tensorflow27/lib/python2.7/site-packages/ipykernel_launcher.py:16: UserWarning: loadtxt: Empty input file: "./trace/0000000001.txt"
  app.launch_new_instance()
/home/tommaso/anaconda3/envs/tensorflow27/lib/python2.7/site-packages/ipykernel_launcher.py:16: UserWarning: loadtxt: Empty input file: "./trace/0000000004.txt"
  app.launch_new_instance()
/home/tommaso/anaconda3/envs/tensorflow27/lib/python2.7/site-packages/ipykernel_launcher.py:16: UserWarning: loadtxt: Empty input file: "./trace/0000000006.txt"
  app.launch_new_instance()
/home/tommaso/anaconda3/envs/tensorflow27/lib/python2.7/site-packages/ipykernel_launcher.py:16: UserWarning: loadtxt: Empty input file: "./trace/0000000007.txt"
  app.launch_new_instance()
/home/tommaso/anaconda3/envs/tensorflow27/lib/python2.7/site-packages/ipykernel_launcher.py:16: UserWarning: loadtxt: Empty input file: "./trace/0000000008.txt"
  app.launch_new_instance()
/home/tommaso/anaconda3/envs/tensorflow27/lib/python2.7

In [174]:
def build_graph(trace):
    prev_item = trace[0]
    for i in range(1,num_pics):
        cur_item = trace[i]
        prev_boxes = get_boxes(trace[i-1])
        cur_boxes = get_boxes(trace[i])
        
        edges = build_edges(prev_boxes,cur_boxes)
        print(edges)
        print(i,'-----------------')
        
        

In [175]:
def get_boxes(items):
    '''Extract boxes from trace items'''
    dims = items.shape
    boxes = []
    if(len(dims) == 1):
        if(dims[0] > 0):
            boxes = np.array([ items[:4] ])
            boxes.reshape((1,4))          
    else:
        for item in items:
            boxes.append( item[:4] )
    return np.array(boxes)

def dist(x,y):
    '''Euclidean dist'''
    return np.linalg.norm(np.array(x) - np.array(y))


In [176]:
def build_edges(prev_boxes,cur_boxes):
    
    prev_dim = prev_boxes.shape
    cur_dim = cur_boxes.shape
    
    edges = []
    EPS = 250
    
    for i in range(cur_dim[0]):
        for j in range(prev_dim[0]):
            if dist(cur_boxes[i],prev_boxes[j]) < EPS:
                edges += [(i,j)]
    return edges
                


In [177]:
build_graph(trace)

[]
1 -----------------
[]
2 -----------------
[(0, 0)]
3 -----------------
[]
4 -----------------
[]
5 -----------------
[]
6 -----------------
[]
7 -----------------
[]
8 -----------------
[]
9 -----------------
[]
10 -----------------
[]
11 -----------------
[]
12 -----------------
[]
13 -----------------
[]
14 -----------------
[]
15 -----------------
[]
16 -----------------
[]
17 -----------------
[]
18 -----------------
[]
19 -----------------
[]
20 -----------------
[]
21 -----------------
[]
22 -----------------
[]
23 -----------------
[]
24 -----------------
[]
25 -----------------
[]
26 -----------------
[]
27 -----------------
[]
28 -----------------
[]
29 -----------------
[]
30 -----------------
[]
31 -----------------
[]
32 -----------------
[]
33 -----------------
[]
34 -----------------
[]
35 -----------------
[]
36 -----------------
[]
37 -----------------
[]
38 -----------------
[]
39 -----------------
[]
40 -----------------
[]
41 -----------------
[]
42 -------------

In [144]:
get_boxes(trace[1]).shape

(0,)

In [101]:
get_boxes(trace[0])

[array([  50.229347,  265.349335,  100.458694,  201.623413])]

In [88]:
type(trace[1])

numpy.ndarray